In [118]:
import numpy as np
import k3d
from tqdm import tqdm

In [119]:
def sphere(N, radius=1, origin=[0,0,0]):
    if np.shape(origin) != (3,):
        raise ValueError('Origin should be list of 3 elements')
    origin = np.float32(origin)
    
    u = np.linspace(0, 2 * np.pi, N, dtype=np.float32)
    v = np.linspace(0, np.pi, N, dtype=np.float32)
    x = radius * np.outer(np.cos(u), np.sin(v))
    y = radius * np.outer(np.sin(u), np.sin(v))
    z = radius * np.outer(np.ones(np.size(u)), np.cos(v)).astype(np.float32)
    x, y, z = np.ravel(x), np.ravel(y), np.ravel(z) 
    xyz = np.vstack([x,y,z]).T + origin
    
    points = k3d.points(positions=xyz, point_size=radius/50)
    return points

def circle(N, radius=1, origin=[0,0,0]):
    if np.shape(origin) != (3,):
        raise ValueError('Origin should be list of 3 elements')
    origin = np.float32(origin)
    
    circle = []
    for i in range(N):
        circle.append([radius*np.cos(2*np.pi*i/N), radius*np.sin(2*np.pi*i/N),0])
    circle = np.array(circle) + origin
    points = k3d.points(positions=np.float32(circle), point_size=radius/50)
    return points
    

def E(q, r0, x, y, z):
    """Return the electric field vector E=(Ex,Ey,Ez) due to charge q at r0."""
    den = np.hypot(x-r0[0], y-r0[1], z-r0[2])**3
    return q * (x - r0[0]) / den, q * (y - r0[1]) / den, q * (z - r0[2]) / den

In [120]:
charge_color = {1:0xff0000, -1:0x0000ff}
charge_color_inv = {0xff0000:1, 0x0000ff:-1}
charges = [(-1, (0.5, -0.5, 0)), (1, (-0.5, -0.5, 0)), (1, (0.5, 0.5, 0)), (-1, (-0.5, 0.5, 0)) ]    

In [121]:
plot = k3d.plot(grid_auto_fit=False, camera_auto_fit=False)
charges_k3d = []

# Add charges to plot
for i in range(len(charges)):
    charges_k3d.append(k3d.points(charges[i][1], color=charge_color[charges[i][0]], point_size=0.1))
    plot += charges_k3d[i]

In [122]:
# Add vector field to plot
nx, ny, nz = 20, 20, 1
x = np.linspace(-2.0, 2.0, nx, dtype=np.float32)
y = np.linspace(-2.0, 2.0, ny, dtype=np.float32)
z = np.linspace(0.0, 2.0, nz, dtype=np.float32)
X, Y, Z = np.meshgrid(x, y, z)
Ex, Ey, Ez = np.zeros((ny, nx, nz)), np.zeros((ny, nx, nz)), np.zeros((ny, nx, nz))
for charge in charges:
    ex, ey, ez = E(*charge, x=X, y=Y, z=Z)
    Ex += ex
    Ey += ey
    Ez += ez

Ex, Ey, Ez = np.ravel(Ex), np.ravel(Ey), np.ravel(Ez)
efield = np.stack([Ex, Ey, Ez]).T 
efield = np.float32(efield / np.stack([np.linalg.norm(efield, axis=1)]).T)/6
X, Y, Z = np.ravel(X), np.ravel(Y), np.ravel(Z)

electric_field  = k3d.vectors(np.stack([X, Y, Z]).T, efield, head_size=0.5)
plot += electric_field

In [123]:
# Create list of k3d objects
test_charges = []
for charge in charges_k3d:
    #test_charges.append(sphere(7, radius=0.25, origin=charge.positions))
    test_charges.append(circle(66, radius=0.25, origin=charge.positions))

In [124]:
# Show objects on plot
for test_charges_sphere in test_charges:
    plot += test_charges_sphere

In [125]:
# Create empty containers for field force lines
trajectories = []
for index, test_charge in enumerate(test_charges):
    trajectories.append([])
    for i in range(len(test_charge.positions)):
        trajectories[index].append([])

In [126]:
# Simulation 
for t in tqdm(range(300)):
    
    # Create empty containers for electric field components in test charges places
    test_charges_efield = []
    for test_charge in test_charges:
        N_test_charges = len(test_charge.positions)
        test_charges_efield.append([np.zeros(N_test_charges), np.zeros(N_test_charges), np.zeros(N_test_charges)])

    # Calculate electric field for each bundle of test charges from each charge
    for charge in charges:
        for index in range(len(charges)):
            X1, Y1, Z1 = test_charges[index].positions[:,0], test_charges[index].positions[:,1], test_charges[index].positions[:,2]
            X1, Y1, Z1 = X1+np.finfo(np.float32).eps, Y1+np.finfo(np.float32).eps, Z1+np.finfo(np.float32).eps 
            ex, ey, ez = E(*charge, x=X1, y=Y1, z=Z1)
            test_charges_efield[index][0] += ex
            test_charges_efield[index][1] += ey
            test_charges_efield[index][2] += ez
        
    dt = np.float32(t/10000)
    
    # Move test charges 
    for i in range(len(test_charges_efield)):
        ET = np.float32(np.vstack(test_charges_efield[i]).T)
        q = charges[i][0]
        test_charges[i].positions = test_charges[i].positions + q*ET*dt

        for j in range(len(trajectories[i])):
            trajectories[i][j].append(test_charges[i].positions[j])

100%|██████████| 300/300 [00:03<00:00, 98.97it/s] 


In [127]:
for i in range(np.shape(trajectories)[0]):
    for j in range(np.shape(trajectories)[1]):
        trajectories[i][j] = np.array(trajectories[i][j])

In [128]:
# Cutting field force lines to box of dimensions
xmax, ymax, zmax =  2.0, 2.0, 2.0
xmin, ymin, zmin = -2.0, -2.0, -2.0
for i in range(np.shape(trajectories)[0]):
    for j in range(np.shape(trajectories)[1]):
        cond1 = (np.sum(trajectories[i][j] < np.array([xmax, ymax, zmax]), axis=1)//3).astype(np.bool)
        cond2 = (np.sum(trajectories[i][j] > np.array([xmin, ymin, zmin]), axis=1)//3).astype(np.bool)
        square_cond = cond1 & cond2
        try:
            border = np.where(square_cond == False)[0][0]
        except IndexError:
            border = None
        trajectories[i][j] = trajectories[i][j][:border]

In [129]:
# Cutting field force lines in the surroundings of charges () 
radius = 0.25
for i in range(np.shape(trajectories)[0]):
    for j in range(np.shape(trajectories)[0]):
        if j==i:
            continue
        for k in range(np.shape(trajectories[i])[0]):
            neg_area_cond = np.sqrt(np.sum((trajectories[i][k] - charges_k3d[j].positions)**2, axis=1)) > radius
            try:
                border = np.where(neg_area_cond == False)[0][0]
            except IndexError:
                border = None
            trajectories[i][k] = trajectories[i][k][:border]

In [130]:
# Adding field force lines to plot
for i in tqdm(range(len(trajectories))):
    for j in range(len(trajectories[0])):
        plot += k3d.line(trajectories[i][j], shader='simple', color=0)

100%|██████████| 4/4 [00:03<00:00,  1.12it/s]


In [131]:
plot.display()

Output()